# An introduction to Keras

<div align="center">
  <img src="https://keras.io/img/logo.png">
</div>

[Keras](https://keras.io/) is an open-source software library that provides a Python interface for artificial neural networks. Keras acts as an interface for the TensorFlow library.
Up until version 2.3 Keras supported multiple backends, including TensorFlow, Microsoft Cognitive Toolkit, R, Theano, and PlaidML. As of version 2.4, only TensorFlow is supported. Designed to enable fast experimentation with deep neural networks, it focuses on being user-friendly, modular, and extensible. 

It was developed as part of the research effort of project ONEIROS (Open-ended Neuro-Electronic Intelligent Robot Operating System), and its primary author and maintainer is François Chollet, a Google engineer.


- Guide: see TensorFlow [Guide](https://www.tensorflow.org/guide) / Keras
- Python API reference: see TensorFlow [Python API reference](https://www.tensorflow.org/api_docs/python/tf/keras)
- [GitHub development repository](https://github.com/keras-team/keras), now migrated into TensorFlow [GitHub development repository](https://github.com/tensorflow/tensorflow)
- [Code examples](https://keras.io/examples/)

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

## MNIST benchmark with Keras

### Preparing data

Load [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset. Notice that TensorFlow uses by default the first axes for samples/batches and the last ones for features.

In [2]:
data = np.genfromtxt('sample_data/mnist_train_small.csv', delimiter=',')
y_data = data[:,0]
x_data = data[:,1:] / 255
print(x_data.shape, "\n")
print(y_data.shape, "\n")

(20000, 784) 

(20000,) 



Train-validation split.

In [3]:
n_train = 10000
n_valid = 10000
x_train = x_data[:n_train ,:]
y_train = y_data[:n_train]
x_valid = x_data[-n_valid:,:]
y_valid = y_data[-n_valid:]

### ANN setup and training

We creare a fully connected ANN, with two 50-neuron hidden layers and a soft-max layer.

In [4]:
model = keras.Sequential([
    keras.layers.Dense(50, activation='tanh', input_shape=(784,)),
    keras.layers.Dense(50, activation='tanh'),
    keras.layers.Dense(10),
    keras.layers.Softmax()
])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                39250     
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
 softmax (Softmax)           (None, 10)                0         
                                                                 
Total params: 42,310
Trainable params: 42,310
Non-trainable params: 0
_________________________________________________________________


We compile the model (that is, we trace the computational graph by [Autograph](https://www.tensorflow.org/guide/intro_to_graphs)).

In [6]:
model.compile(optimizer = keras.optimizers.RMSprop(learning_rate=0.002,rho=0.9),
              loss = keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

We train the model.

In [7]:
model.fit(x_train, y_train, epochs = 50, validation_data = (x_valid, y_valid), batch_size=1000)

Epoch 1/50
10/10 [==============================] - 1s 34ms/step - loss: 1.3722 - accuracy: 0.5944 - val_loss: 0.8565 - val_accuracy: 0.7996
Epoch 2/50
10/10 [==============================] - 0s 17ms/step - loss: 0.7300 - accuracy: 0.8221 - val_loss: 0.6038 - val_accuracy: 0.8543
Epoch 3/50
10/10 [==============================] - 0s 18ms/step - loss: 0.5388 - accuracy: 0.8626 - val_loss: 0.4832 - val_accuracy: 0.8783
Epoch 4/50
10/10 [==============================] - 0s 16ms/step - loss: 0.4350 - accuracy: 0.8896 - val_loss: 0.4047 - val_accuracy: 0.8958
Epoch 5/50
10/10 [==============================] - 0s 18ms/step - loss: 0.3835 - accuracy: 0.8951 - val_loss: 0.4061 - val_accuracy: 0.8780
Epoch 6/50
10/10 [==============================] - 0s 16ms/step - loss: 0.3328 - accuracy: 0.9082 - val_loss: 0.3549 - val_accuracy: 0.8992
Epoch 7/50
10/10 [==============================] - 0s 17ms/step - loss: 0.3094 - accuracy: 0.9121 - val_loss: 0.3089 - val_accuracy: 0.9121
Epoch 8/50
10

### Testing

We load testing data.

In [8]:
data_test = np.genfromtxt('sample_data/mnist_test.csv', delimiter=',')
y_test = data_test[:,0]
x_test = data_test[:,1:] / 255
print(x_test.shape, "\n")
print(y_test.shape, "\n")

(10000, 784) 

(10000,) 



We evaluate the accuracy of the model in the test dataset.

In [9]:
predicted_probabilities = model.predict(x_test)
predicted_labels = tf.argmax(predicted_probabilities, axis=1)
accuracy_test = tf.reduce_mean(tf.cast(predicted_labels == y_test, tf.float32))
print('Accuracy (test dataset): %1.2f%%' % (accuracy_test * 100))

Accuracy (test dataset): 94.75%


Equivalent implementation (using the built-in metric).

In [10]:
accuracy_test = keras.metrics.SparseCategoricalAccuracy()(y_test, model.predict(x_test))
print('Accuracy (test dataset): %1.2f%%' % (accuracy_test * 100))

Accuracy (test dataset): 94.75%


## Writing a custom training loop

Keras adopts the principle of progressive disclosure of complexity: simple workflows should be quick and easy, while arbitrarily advanced workflows should be possible via a clear path that builds upon what you've already learned.

In [11]:
batch_size = 250
num_epochs = 20

optimizer = keras.optimizers.RMSprop(learning_rate=0.002,rho=0.9)
loss = keras.losses.SparseCategoricalCrossentropy()

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(784,)),
    keras.layers.Dense(50, activation='tanh'),
    keras.layers.Dense(50, activation='tanh'),
    keras.layers.Dense(10),
    keras.layers.Softmax()
])

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))

for epoch in range(num_epochs):
  for (x_batch, y_batch) in dataset.batch(batch_size):
    with tf.GradientTape() as tape:
      y_predicted_batch = model(x_batch)
      current_loss = loss(y_batch, y_predicted_batch)
    gradients = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  accuracy_train = keras.metrics.SparseCategoricalAccuracy()(y_train, model.predict(x_train))
  accuracy_valid = keras.metrics.SparseCategoricalAccuracy()(y_valid, model.predict(x_valid))
  print('epoch %2d - accuracy: train %1.2f%%, valid %1.2f%%' % (epoch + 1, accuracy_train * 100, accuracy_valid * 100))

epoch  1 - accuracy: train 87.51%, valid 87.68%
epoch  2 - accuracy: train 90.43%, valid 90.28%
epoch  3 - accuracy: train 92.02%, valid 91.07%
epoch  4 - accuracy: train 93.08%, valid 91.57%
epoch  5 - accuracy: train 93.70%, valid 92.12%
epoch  6 - accuracy: train 94.50%, valid 92.42%
epoch  7 - accuracy: train 95.18%, valid 92.72%
epoch  8 - accuracy: train 95.85%, valid 93.00%
epoch  9 - accuracy: train 96.45%, valid 93.13%
epoch 10 - accuracy: train 96.79%, valid 93.34%
epoch 11 - accuracy: train 97.23%, valid 93.42%
epoch 12 - accuracy: train 97.69%, valid 93.54%
epoch 13 - accuracy: train 98.12%, valid 93.63%
epoch 14 - accuracy: train 98.45%, valid 93.95%
epoch 15 - accuracy: train 98.73%, valid 94.15%
epoch 16 - accuracy: train 99.03%, valid 94.32%
epoch 17 - accuracy: train 99.26%, valid 94.41%
epoch 18 - accuracy: train 99.41%, valid 94.52%
epoch 19 - accuracy: train 99.52%, valid 94.55%
epoch 20 - accuracy: train 99.48%, valid 94.25%


In [12]:
print('Accuracy (train dataset)     : %1.2f%%' % (keras.metrics.SparseCategoricalAccuracy()(y_train, model.predict(x_train)) * 100))
print('Accuracy (validation dataset): %1.2f%%' % (keras.metrics.SparseCategoricalAccuracy()(y_valid, model.predict(x_valid)) * 100))
print('Accuracy (test dataset)      : %1.2f%%' % (keras.metrics.SparseCategoricalAccuracy()(y_test,  model.predict(x_test )) * 100))

Accuracy (train dataset)     : 99.48%
Accuracy (validation dataset): 94.25%
Accuracy (test dataset)      : 94.63%
